In [9]:
"""
Convert GloVe file to word2vec file
"""

import os
from gensim.scripts.glove2word2vec import glove2word2vec

dirname = os.path.abspath('')
glove_input_file = os.path.join(dirname, '..', '..', 'data', 'glove.6B.300d.txt')
word2vec_output_file = os.path.join(dirname, '..', '..', 'data', 'word2vec.300d.txt')

glove2word2vec(glove_input_file, word2vec_output_file)


C:\Users\marcus.joyce\AppData\Local\Temp\ipykernel_76072\4204654206.py:12: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


(400000, 300)

In [ ]:
"""
Load the word2vec model
"""

from gensim.models import KeyedVectors
import os

dirname = os.path.abspath('')
filename = os.path.join(dirname, '..', '..', 'data', 'word2vec.100d.txt')

model = KeyedVectors.load_word2vec_format(filename, binary=False)

In [10]:
"""
Loads file
"""

import numpy as np
import pandas as pd
import os

dirname = os.path.abspath('')
data = os.path.join(dirname, '..', '..', 'data', 'all_data_processed.csv')

df = pd.read_csv(data)

'''
n_datapoints = 100000

df = df[:n_datapoints]
'''

max_sentence_length = 300
dimension = 100


def vectorize(sentence):
    """
    Takes a list of words and returns a list of vectors through word embeddings.
    """

    out = np.empty((max_sentence_length, dimension))
    i_ = 0
    for i, word in enumerate(sentence):
        if i < max_sentence_length:
            try:
                out[i] = model[word]
            except KeyError:
                out[i] = np.zeros(dimension)
            i_ += 1
    out[range(i_+1, max_sentence_length)] = np.zeros(dimension)  # pad the array with arrays of zeros.

    return out


In [11]:
"""
Take the average of toxicity and severe_toxicity and save file.
"""

import os
dirname = os.path.abspath('')

scores = df[['toxicity', 'severe_toxicity']]

scores_np = np.empty(len(scores), dtype='f')

for i, row in scores.iterrows():
    scores_np[i] = (row[0] + row[1])/2

del scores

np.save(os.path.join(dirname, '..', '..', 'data', 'y.npy'), scores_np)

del scores_np

In [5]:
"""
Vectorize every word in the dataset and save file.
"""

import os
dirname = os.path.abspath('')

comments = df['comment_text'].apply(eval)  # The lists in the dataframe are stored as strings and need to be converted to lists.

comments_vectorized = np.empty((len(comments), max_sentence_length, dimension), dtype='f')

for j, c in enumerate(comments):
    comments_vectorized[j] = vectorize(c)

del comments

np.save(os.path.join(dirname, '..', '..', 'data', 'X.npy'), comments_vectorized)

del comments_vectorized
